In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [ ]:
#DEFINE data generators for train,validation and test sets
train_datagen=ImageDataGenerator(
    rescale=1./225,
    validation_split=0.2
)
train_generator=train_datagen.flow_from_directory("/content/drive/MyDrive/Dataset/Brain_Tumor_Detection/train",
              target_size=(IMG_SIZE,IMG_SIZE),
              batch_size=BATCH_SIZE,
              class_mode='binary',
              subset='training'
)
val_generator=train_datagen.flow_from_directory("/content/drive/MyDrive/Dataset/Brain_Tumor_Detection/train",
              target_size=(IMG_SIZE,IMG_SIZE),
              batch_size=BATCH_SIZE,
              class_mode='binary',
              subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./225)
test_generator=test_datagen.flow_from_directory("/content/drive/MyDrive/Dataset/Brain_Tumor_Detection/test",
              target_size=(IMG_SIZE,IMG_SIZE),
              batch_size=BATCH_SIZE,
              class_mode='binary',
)

Found 2416 images belonging to 2 classes.
Found 604 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [ ]:
#define the model
model=keras.Sequential([
      layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
      layers.MaxPooling2D((2,2)),
      layers.Conv2D(64,(3,3),activation='relu'),
      layers.MaxPooling2D((2,2)),
      layers.Conv2D(128,(3,3),activation='relu'),
      layers.MaxPooling2D((2,2)),
      layers.Flatten(),
      layers.Dense(128,activation='relu'),
      layers.Dense(1,activation='sigmoid')
])

In [ ]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
76/76 [==============================] - 492s 6s/step - loss: 0.4522 - accuracy: 0.8001 - val_loss: 0.2647 - val_accuracy: 0.8858
Epoch 2/5
76/76 [==============================] - 317s 4s/step - loss: 0.2030 - accuracy: 0.9251 - val_loss: 0.1399 - val_accuracy: 0.9553
Epoch 3/5
76/76 [==============================] - 327s 4s/step - loss: 0.1024 - accuracy: 0.9661 - val_loss: 0.0508 - val_accuracy: 0.9818
Epoch 4/5
76/76 [==============================] - 327s 4s/step - loss: 0.0380 - accuracy: 0.9901 - val_loss: 0.0857 - val_accuracy: 0.9603
Epoch 5/5
76/76 [==============================] - 313s 4s/step - loss: 0.0256 - accuracy: 0.9909 - val_loss: 0.0405 - val_accuracy: 0.9834


In [ ]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
#load the saved model
model=load_model('/content/drive/MyDrive/Model.h5')
#load and preprocess the test image
test_image_path='/content/drive/MyDrive/Dataset/Brain_Tumor_Detection/train/yes/y10.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)  #add batch dimesion
img_array /=225. #normalize the pixel values
#make predictions
predicition=model.predict(img_array)
#print the prediction
if predicition < 0.5:
  print("predicition: No tumor (Probability:",predicition[0][0],")")
else:
  print("predicition: Tumor prediction (Probability:",predicition[0][0],")")

1/1 [==============================] - 0s 204ms/step
predicition: Tumor prediction (Probability: 0.99999976 )
